In [1]:
import pandas as pd
import numpy as np
import urllib3
import certifi
import tqdm
import BasicFunctions as bf
import DataCorrecting as dc
import json

from lxml import etree
from io import StringIO, BytesIO
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [2]:
train, test = bf.data_extract('train_indexes.csv', 'test_indexes.csv')

In [4]:
def pc_from_adress(X):
    postal_codes = X.postal_code.values
    for i, address in zip(range(X.shape[0]), X.address_rus):
        postal_code = address.split()[-1]
        if postal_code.isdigit():
            postal_codes[i] = postal_code
    X.postal_code = postal_codes
    return X

In [5]:
train = pc_from_adress(train)
test = pc_from_adress(test)

In [6]:
def pc_from_net(X):
    driver = webdriver.Chrome()
    driver.get('https://www.pochta.ru/post-index')
    indexes = X.postal_code.values
    for i in tqdm.tqdm_notebook(range(X.shape[0])):# - 1, 0, -1)):
        if X.postal_code[i] == 0:
            try:
                address_text = X.address_rus[i]
                address = driver.find_element_by_xpath("//*[@type='text']")
                address.send_keys(address_text)
                address.send_keys(Keys.ENTER)
                time.sleep(1)
                try:
                    index = driver.find_element_by_css_selector(".index-card__postal-code")
                    indexes[i] = index.text
                except BaseException:
                    indexes[i] = 0
                driver.refresh()
            except BaseException:
                print('Error', i)
                time.sleep(10)
                driver.refresh()
    X.postal_codes = indexes
    return X

In [18]:
new_test = pc_from_net(new_test)
new_train = pc_from_net(new_train)